# Download ERA5 bi-weekly data using a grid

In [1]:
import ee, geemap, os, time
import numpy as np

In [2]:
print(ee.__version__)
print(geemap.__version__)
ee.Initialize() # <- must do

0.1.221
0.6.13


## 1. Set parameters

In [33]:
# set region
COI = 'Costa Rica'
SOI = ''
# set number of tiles you want to download
IDX = 0
IDX_STEP = 20
# set date range for temp/precip data
SDATE = '2017-01-01'
EDATE = '2020-12-31'
# set parameters for calculating 15-day step
STEP = 15
SYEAR = 2017
EYEAR = 2020
# choose which data to export (temp or precip) and where
EXPORT_NAME = 'amazonas_biweekly_temp'
OUT_DIR = "/Users/morgansteckler/Desktop"
SCALE = 20

## 2. Access admin data and create grid
- This block of code accesses global admin data (https://developers.google.com/earth-engine/datasets/catalog/FAO_GAUL_2015_level1) where I can get Amazonas as a feature. I then overlay my grid and extract the grid coordinates to create a simple feature collection. 
- Here, I also select the number of tiles to export. I usually set it to 50 at a time (I have four GEE accounts to get around export limitations, so I can actually do 200 at a time). 

In [34]:
# select data bounds
br = (ee.FeatureCollection("FAO/GAUL/2015/level1")
       .filterMetadata('ADM0_NAME', 'equals', COI)
       #.filterMetadata('ADM1_NAME', 'equals', SOI)
      )

# set up grid process
def make_grid(region, a_scale):
    lonLat = ee.Image.pixelLonLat()

    lonGrid = (lonLat.select('latitude').multiply(10000000).toInt())
    latGrid = (lonLat.select('longitude').multiply(10000000).toInt())

    grid = (lonGrid.multiply(latGrid).reduceToVectors(
        geometry = region, 
        scale = a_scale, 
        geometryType = 'polygon'))
    
    return(grid)

# make grid overtop Amazonas and set gridsize (40,000 = 40km)
grid_km = make_grid(br, 25000)

# access coordinates of grid squares
grid_dict = grid_km.getInfo()
feats = grid_dict['features']
coord_list = []
for d in feats:
    geom = d['geometry']
    coords = geom['coordinates']
    coord_list.append(coords)
    
# create a list of several ee.Geometry.Polygons
polys = []
for coord in coord_list:
    poly = ee.Geometry.Polygon(coord)
    polys.append(poly)
print("There are {} cells in the entire grid.".format(str(len(polys))))
    
# select range of data for export
NEX = IDX + IDX_STEP
polys = [polys[i] for i in list(range(IDX,NEX))]

# make the whole grid a feature collection for export purposes
grid = ee.FeatureCollection(polys)
fc = grid

There are 86 cells in the entire grid.


## 3. Visualize just for fun
This is using a package called geemap, develed by Dr. Wu at UTK Geography.
https://github.com/giswqs/geemap

In [35]:
# visualize the grid feature collection and amazonas feature
Map = geemap.Map()
Map.add_basemap("SATELLITE")
Map.addLayer(fc, {}, (SOI + ' Grid AOI'))
Map.addLayer(br, {}, SOI)
Map.center_object(br, zoom=5)
Map

Map(center=[9.972812373768427, -84.19347024305382], controls=(WidgetControl(options=['position'], widget=HBox(…

## 4. Access ERA-5 data
https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_DAILY

In [36]:
# get daily mean air temp (K) at .25 arc degree res
era5_2mt = (ee.ImageCollection('ECMWF/ERA5/DAILY')
            .select('mean_2m_air_temperature')
            .filter(ee.Filter.date(SDATE, EDATE))
           )

# get total precipitation (m) at .25 arc degree res
era5_tp = (ee.ImageCollection('ECMWF/ERA5/DAILY')
            .select('total_precipitation')
            .filter(ee.Filter.date(SDATE, EDATE))
          )

# get hourly surface_solar_radiation_downwards at .1 arc degree res
era5_srd = (ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')
           .select('surface_solar_radiation_downwards')
           .filter(ee.Filter.date(SDATE, EDATE))
           )

In [37]:
# Select your data of interest
DOI = era5_srd

In [38]:
DOI.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'surface_solar_radiation_downwards',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [3601, 1801],
   'crs': 'EPSG:4326',
   'crs_transform': [0.1, 0, -180.05, 0, -0.1, 90.05]}],
 'version': 1618811718668955,
 'id': 'ECMWF/ERA5_LAND/HOURLY/20170101T00',
 'properties': {'system:time_start': 1483228800000,
  'hour': 0,
  'system:footprint': {'type': 'LinearRing',
   'coordinates': [[-180, -90],
    [180, -90],
    [180, 90],
    [-180, 90],
    [-180, -90]]},
  'system:time_end': 1483232400000,
  'system:asset_size': 402009425,
  'system:index': '20170101T00'}}

In [39]:
# clip data to grid cells
clipped_cols = []
for poly in polys:
    clipped_col = DOI.map(lambda image: image.clip(poly))
    clipped_cols.append(clipped_col)

## 5. Create 15-day step
- This block of code finds the mean temperature for 15-day steps and creates a new property for each image called "step". I then print the properties to make sure the steps are added.

In [45]:
# apply 15-day step
years = ee.List.sequence(2017, 2017)
step = ee.List.sequence(1, 365, STEP)

all_cols = []
for a_col in clipped_cols:
    def byYear(y):
        y = ee.Number(y)
        def byStep(d):
            d = ee.Number(d)
            return (a_col
                    .filter(ee.Filter.calendarRange(y, y, 'year'))
                    .filter(ee.Filter.calendarRange(d, d.add(14), 'day_of_year'))
                    .mean()
                    .set('step', [d, y]))
                
        return step.map(byStep)

    col = ee.ImageCollection.fromImages(years.map(byYear).flatten())
    all_cols.append(col)

# make sure 'properties' are correct
column = all_cols[1].filter(ee.Filter.eq('system:index', '0'))
print(column.getInfo())

{'type': 'ImageCollection', 'bands': [], 'features': [{'type': 'Image', 'bands': [{'id': 'surface_solar_radiation_downwards', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'step': [1, 2017], 'system:index': '0'}}]}


In [44]:
print(all_cols[0].size().getInfo())

25


In [46]:
print(all_cols[0].first().propertyNames().getInfo())

['step', 'system:index', 'system:bands', 'system:band_names']


## 6. Export data
- The final step, exporting. We loop through the images, tile names, and tile polygons. We check to make sure data exists (this avoids a fatal GEE error), we create a new directory for export, and we skip images that have already been downloaded. So, if you've already got data under the same name, delete it beforehand. 

- NOTE: Since the data is coarse resolution, I shouldn't hit any GEE error about maxing out my download abilities. For sentinel, I implemented a try and except method here to ignore GEE errors and keep running.

In [9]:
# make a list of file names for exporting
tiles = []
EXPORT_NAME = 'amazonas_biweekly_temp'
for num in range(IDX,NEX):
    index = str(EXPORT_NAME + '_{}'.format(num))
    tiles.append(index)

print("Tiles {} to {} will be exported for {}.".format(IDX,NEX-1,SOI))

Tiles 0 to 49 will be exported for Amazonas.


In [10]:
# export and keep time
tic1 = time.time()
for a_col, a_tile, poly in zip(all_cols, tiles, polys):
    ilist = a_col.toList(a_col.size())
    for i in range(0,100):
        if len(ee.Image(ilist.get(i)).bandNames().getInfo()) <= 0:
            print("No bands found in image {}. Skipping...".format(i))
        else:
            filename = "{}/{}/{}.tif".format(OUT_DIR,a_tile,i)
            temp_dir = "{}/{}/".format(OUT_DIR,a_tile)
            if not os.path.exists(temp_dir):
                os.mkdir(temp_dir)
            if os.path.exists(filename):
                print("Image {} already exists. Skipping...".format(i))
                next
            else:
                print("Exporting Image {}...".format(i))
                geemap.ee_export_image(ee.Image(ilist.get(i)), 
                                       filename=filename, 
                                       scale=SCALE, 
                                       region=poly, 
                                       file_per_band=False)
toc1 = time.time()
hrs1, rem1 = divmod(toc1-tic1, 3600)
mins1, secs1 = divmod(rem1,  60)
print("Total time elapsed: {:0>2}:{:0>2}:{:05.2f}"
      .format(int(hrs1),int(mins1),secs1))

Exporting Image 0
Generating URL ...
Please wait ...
Data downloaded to /Users/morgansteckler/Desktop/amazonas_biweekly_temp_0/0.tif
Exporting Image 1
Generating URL ...
Please wait ...
Data downloaded to /Users/morgansteckler/Desktop/amazonas_biweekly_temp_0/1.tif
Exporting Image 2
Generating URL ...
Please wait ...


KeyboardInterrupt: 